In [ ]:
!pip install natasha

In [2]:
import pandas as pd
from natasha import NamesExtractor
import pymorphy2
from natasha import MorphVocab

In [ ]:
data = pd.read_csv('test_data.csv')
data.head()

In [ ]:
data.info()

In [ ]:
# создание поля для флагов реплик
data['flags'] = ''
data

In [6]:
voc = MorphVocab()
ext = NamesExtractor(voc)

In [ ]:
for i in range(data['dlg_id'].max()+1):
  greeting = False # флаг приветствия менеджера в диалоге
  self_pres = False # флаг представления менеджера в реплике
  name_pres = False # флаг имени менеджера в реплике
  company_name = False # флаг названия компании в реплике
  goodbye = False # флаг прощания менеджера в диалоге
  print('Диалог №'+str(i))
  slice_data = data[(data['dlg_id']==i)&(data['role']=='manager')]

  # проверка приветствия, представления, названия компании и имени в первых 5 фразах
  # взято с запасом, т.к. могут быть 3 однотипных фразы 'ало'
  # при попытке связаться с клиентом 
  slice_greeting = slice_data[:5]
  for piece in slice_greeting['text']:
    if not greeting:
      if any([x in piece.lower() for x in ('здравствуйте','добрый','добрый день',
                                           'доброе утро', 'утро доброе', 'доброе',
                                           'день добрый', 'приветствую', 'доброй',
                                           'добрый вечер', 'вечер добрый',
                                           'доброй ночи', 'ночи', 'ночи доброй',
                                           'доброго времени суток')]):
        print('Реплика с приветствием: ', piece)
        data.loc[(data['text']==piece)&(data['dlg_id']==i)&(data['role']=='manager'), 'flags'] += 'greeting=True,'
        greeting = True
    
    if not self_pres:
      if any([x in piece.lower() for x in ('меня зовут','я представляю компанию',
                                           'я представляю интерес компании',
                                           'я менеджер компании',
                                           'я представитель компании')]):
        print('Реплика с представлением: ', piece)
        data.loc[(data['text']==piece)&(data['dlg_id']==i)&(data['role']=='manager'), 'flags'] += 'self_pres=True,'
        self_pres = True

    if not name_pres:
      text_greeting = piece.strip()
      matches = ext(text_greeting)
      name = 'unknown'
      for index, match in enumerate(matches):
        if match.fact.first != None:
          name = str(match.fact.first)
      if any([x in piece.lower() for x in ('меня зовут','мое имя','зовут меня',
                                           'меня '+name+' зовут', 'имя '+name+' мое',
                                           'звонит '+name, 'беспокоит '+name,
                                           name+' беспокоит', 'это '+name)]):
        print('Реплика с именем: '+piece+' | Имя менеджера: '+name)
        data.loc[(data['text']==piece)&(data['dlg_id']==i)&(data['role']=='manager'), 'flags'] += 'name_pres=True,'
        name_pres = True


    # извлечь название компании без нейросети - (?)
    if not company_name:
      if 'компания' in piece.lower():
        print('Реплика с названием компании: ', piece)
        data.loc[(data['text']==piece)&(data['dlg_id']==i)&(data['role']=='manager'), 'flags'] += 'company_name=True,'
        company_name = True

  # проверка прощания в последних 5-ти фразах
  slice_goodbye = slice_data[-5:]
  for piece in slice_goodbye['text']:
    if not goodbye:
      if any([x in piece.lower() for x in ('до свидания','прощайте','доброго дня',
                                            'доброго вечера', 'до связи', 'всего хорошего',
                                            'хорошего дня', 'хорошего вечера', 'всего доброго')]):
        print('Реплика с прощанием: ', piece)
        data.loc[(data['text']==piece)&(data['dlg_id']==i)&(data['role']=='manager'), 'flags'] = 'goodbye=True,'
        goodbye = True

  print()

In [8]:
data['flags'] = data['flags'].apply(lambda x: x.strip(','))

In [ ]:
# все фразы из файла с ненулевыми флагами
data[data['flags'].apply(lambda x: x if x!='' else 0)!=0]